In [1]:
import syft as sy
import torch as t
import numpy as np
import pandas as pd

## Launch a Duet Sever

In [2]:
duet = sy.launch_duet(loopback = True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!

Epoch: 0  Loss: 0.6862443685531616: 4  Requests: 0   Messages: 6  Request Handlers: 1                                
Epoch: 500  Loss: 0.510845959186554 4  Requests: 0   Messages: 6  Request Handlers: 1                                
Epoch: 1500  Loss: 0.4834221303462982  Requests: 0   Messages: 6  Request Handlers: 1                                
Epoch: 3000  Loss: 0.4516996741294861  Requests:

## Read Data and Preprocess

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_2.csv")  

In [4]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [5]:
# Trnasform the input to tensor
X_train_tensor = t.FloatTensor(X_train)
y_train_tensor = t.tensor(y_train,dtype=t.long)
X_test_tensor = t.FloatTensor(X_test)
y_test_tensor = t.tensor(y_test,dtype=t.long)

## Send data to Duet server

In [6]:
X_train_tensor.tag("X_Train_Owner2")
X_train_tensor.describe("Dataset of 400 samples, 7 feature")
X_train_ptr = X_train_tensor.send(duet,pointable = True)

y_train_tensor.tag("y_Train_Owner2")
y_train_tensor.describe("Dataset of 400 samples, 1 feature")
y_train_ptr = y_train_tensor.send(duet,pointable = True)

X_test_tensor.tag("X_Test_Owner2")
X_test_tensor.describe("Dataset of 400 samples, 7 feature")
X_test_ptr = X_test_tensor.send(duet,pointable = True)

y_test_tensor.tag("y_Test_Owner2")
y_test_tensor.describe("Dataset of 400 samples, 1 feature")
y_test_ptr = y_test_tensor.send(duet,pointable = True)


## Create a request handler

In [7]:
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)

In [8]:
duet.store.pandas

ID              Tags  \
0  <UID: 7083551ae5334fc99a22f4b43d44c9b7>  [X_Train_Owner2]   
1  <UID: a45df775e0744a169e1fbbfa7fcbdc03>  [y_Train_Owner2]   
2  <UID: 030c2da760bd4e9e8cd32f19c380988d>   [X_Test_Owner2]   
3  <UID: 245b75ac7b1648f59352636b13d7044f>   [y_Test_Owner2]   

                         Description             object_type  
0  Dataset of 400 samples, 7 feature  <class 'torch.Tensor'>  
1  Dataset of 400 samples, 1 feature  <class 'torch.Tensor'>  
2  Dataset of 400 samples, 7 feature  <class 'torch.Tensor'>  
3  Dataset of 400 samples, 1 feature  <class 'torch.Tensor'>

## Build a local Classifier

In [9]:
class Net(t.nn.Module):
    def __init__(self,input_channels,output_channels):
        super(Net,self).__init__()
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = t.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = t.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = t.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = t.nn.ReLU()
        self.dropout = t.nn.Dropout(p=0.1)
        self.batchnorm1 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = t.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = t.argmax(output,1)
        return prediction
        


In [10]:
def classifier_train(epochs, model, optimizer, X,y,criterion):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred = model.forward(X)
        #Compute Cross entropy loss
        loss = criterion(y_pred,y)
        #Add loss to the list
        losses.append(loss.item())
        
        #Print loss
        if i%500==0:
            print("Epoch:",i," Loss:",loss.item())
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses


## Local Training

In [14]:
# Define the input and output
input_channels = X_train_tensor.shape[1]
output_channels = 2

#Initialize the model        
model = Net(input_channels,output_channels)
#Define loss criterion
criterion = t.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
#Number of epochs
epochs = 5000

losses = classifier_train(
    epochs, model, optimizer,X_train_tensor,y_train_tensor,criterion
)

Epoch: 0  Loss: 0.7021042704582214
Epoch: 500  Loss: 0.4894362688064575
Epoch: 1000  Loss: 0.44651949405670166
Epoch: 1500  Loss: 0.4327508807182312
Epoch: 2000  Loss: 0.42208465933799744
Epoch: 2500  Loss: 0.40434974431991577
Epoch: 3000  Loss: 0.4119190275669098
Epoch: 4500  Loss: 0.3979027271270752


## Accuracy on test set

In [15]:
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [16]:
accuracy(model,X_test_tensor,y_test_tensor)

0.6


[2022-06-21T22:18:30.146857-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 251c8ca1c5ed427c9427966e0312d76c>: To evaluate training progress
Value: <Storable: 0.6677805185317993>
[2022-06-21T22:18:31.169227-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 743c411ad68a4c2cb8979c1ddcfe9576>: To evaluate training progress
Value: <Storable: 0.6677635908126831>
[2022-06-21T22:18:31.331228-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 0397a2058404439283187fd5301d8f16>: To evaluate training progress
Value: <Storable: 0.6663240194320679>
[2022-06-21T22:18:32.471727-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 015c5fb4fb6b4aecb85a6db8045a6b5f>: To evaluate training progress
Value: <Storable: 0.6634174585342407>
[2022-06-21T22:18:32.629707-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: ad43364511ed47c59bc408da0d5d5f11>: To evaluate training progress
Value: <Storable: 0.6648886203765869>
[2022-06-21T22:18:32.788707-0400][CRITICAL][logger]][31540] > HAN

[2022-06-21T22:18:41.033516-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 420f30e735624720ab74ae172d1d2472>: To evaluate training progress
Value: <Storable: 0.6401179432868958>
[2022-06-21T22:18:41.191535-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 4b4d4ce0601344379669f6800c856c85>: To evaluate training progress
Value: <Storable: 0.6393246054649353>
[2022-06-21T22:18:41.342535-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 55f7624403124ffaa41ee350dd7705d1>: To evaluate training progress
Value: <Storable: 0.6381781101226807>
[2022-06-21T22:18:41.501613-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: ba27238c65a540b59f848426b0401eb8>: To evaluate training progress
Value: <Storable: 0.6375691294670105>
[2022-06-21T22:18:41.666066-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 1272fee06151483faeb3ef838e82b33e>: To evaluate training progress
Value: <Storable: 0.6361256241798401>
[2022-06-21T22:18:41.831772-0400][CRITICAL][logger]][31540] > HAN

[2022-06-21T22:18:49.040079-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 6adaa002f8e04bc48ddcdc073f35ae3a>: To evaluate training progress
Value: <Storable: 0.615510880947113>
[2022-06-21T22:18:49.199071-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 95f0926c22b64b87aa9eea319218eabc>: To evaluate training progress
Value: <Storable: 0.611524224281311>
[2022-06-21T22:18:49.361074-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: e3e903e5fab947c090fa81386ebb5d37>: To evaluate training progress
Value: <Storable: 0.6126167178153992>
[2022-06-21T22:18:49.514076-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 3abe7cc157b14057b0a1e41d75679d5c>: To evaluate training progress
Value: <Storable: 0.6115943789482117>
[2022-06-21T22:18:49.678077-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: c046bcc6487949c4aebeb6b37fb55f7f>: To evaluate training progress
Value: <Storable: 0.6096034049987793>
[2022-06-21T22:18:50.817847-0400][CRITICAL][logger]][31540] > HANDL

[2022-06-21T22:18:57.044055-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 31a146b1736a4b74861c6a94f18122f3>: To evaluate training progress
Value: <Storable: 0.5916064977645874>
[2022-06-21T22:18:57.216748-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 17801d5a0d8a4dc5be8682b004cfc93c>: To evaluate training progress
Value: <Storable: 0.5896465182304382>
[2022-06-21T22:18:57.379748-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 67761066c6f641148ca074c34a5a82ac>: To evaluate training progress
Value: <Storable: 0.5893939137458801>
[2022-06-21T22:18:57.533642-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 2cac24c42b104cf7b143d3fe259d8b87>: To evaluate training progress
Value: <Storable: 0.5887174010276794>
[2022-06-21T22:18:57.699356-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 5fece48486ff440fb6317ec91ba5afc9>: To evaluate training progress
Value: <Storable: 0.5887264609336853>
[2022-06-21T22:18:57.842683-0400][CRITICAL][logger]][31540] > HAN

...

[2022-06-21T22:21:03.850480-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 51f2a57d6bf74e17ae9052659e1e9302>: To evaluate training progress
Value: <Storable: 0.48128876090049744>
[2022-06-21T22:21:04.013187-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 62e9756e99de4672999102e5611f7055>: To evaluate training progress
Value: <Storable: 0.482498437166214>
[2022-06-21T22:21:04.180314-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: da10d9e0f58244b68674d117a68d8f22>: To evaluate training progress
Value: <Storable: 0.4740539491176605>
[2022-06-21T22:21:04.329704-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 6f160ec3770d4d739582c633e2fec693>: To evaluate training progress
Value: <Storable: 0.48087096214294434>
[2022-06-21T22:21:04.492761-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: a503f7599286455ab17ea0d5355dc06e>: To evaluate training progress
Value: <Storable: 0.4815269410610199>
[2022-06-21T22:21:04.649719-0400][CRITICAL][logger]][31540] > HA

[2022-06-21T22:21:11.709574-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: d7cd9e62367f452096b1720e657e6f14>: To evaluate training progress
Value: <Storable: 0.47571656107902527>
[2022-06-21T22:21:11.861150-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 28f050f848bc4c75bfb2521680597e7c>: To evaluate training progress
Value: <Storable: 0.4802454113960266>
[2022-06-21T22:21:12.011819-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: c864486841bc4613ac316e9dc3734ca6>: To evaluate training progress
Value: <Storable: 0.4818049669265747>
[2022-06-21T22:21:12.180477-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: beb5b661764144d5bb740007c9994218>: To evaluate training progress
Value: <Storable: 0.47605007886886597>
[2022-06-21T22:21:12.345842-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: f3704e5472544d4a8083d5b88a07cff9>: To evaluate training progress
Value: <Storable: 0.47481533885002136>
[2022-06-21T22:21:12.476428-0400][CRITICAL][logger]][31540] > 

[2022-06-21T22:21:20.597517-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 0863737a729e407eb785c9f78f955249>: To evaluate training progress
Value: <Storable: 0.4735325872898102>
[2022-06-21T22:21:20.761236-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: d8366811acbf4a3c882051d4821b4ba6>: To evaluate training progress
Value: <Storable: 0.4724190831184387>
[2022-06-21T22:21:20.911152-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 2b81bd91fab94429a7995ad3ad1fad6f>: To evaluate training progress
Value: <Storable: 0.4829941689968109>
[2022-06-21T22:21:21.053584-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 1143313f9448417f84663f18b7688389>: To evaluate training progress
Value: <Storable: 0.47300124168395996>
[2022-06-21T22:21:21.216430-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: b5e3bd193c5d417caf7714ec2891d417>: To evaluate training progress
Value: <Storable: 0.4828593134880066>
[2022-06-21T22:21:21.377120-0400][CRITICAL][logger]][31540] > HA

[2022-06-21T22:21:27.503191-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 263e240f0c4645f28407aa180b138016>: To evaluate training progress
Value: <Storable: 0.4738542437553406>
[2022-06-21T22:21:27.663189-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 44768cb54e2f4233ab15b25d69ba06ff>: To evaluate training progress
Value: <Storable: 0.47008216381073>
[2022-06-21T22:21:27.814700-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 2f2f10589faf4322b637d56edce7b86b>: To evaluate training progress
Value: <Storable: 0.4695974290370941>
[2022-06-21T22:21:27.974838-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 65c34a03f0ec491d8bebd45689c2f4ac>: To evaluate training progress
Value: <Storable: 0.47859230637550354>
[2022-06-21T22:21:28.131805-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 46f4bad17b7d48a9876ceadbf890fce0>: To evaluate training progress
Value: <Storable: 0.46820953488349915>
[2022-06-21T22:21:28.298014-0400][CRITICAL][logger]][31540] > HAN

[2022-06-21T23:06:08.716082-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 4c5290ce88284c419f9352424b980015>:
Value: <Storable: 'in_features=7, out_features=10, bias=True'>
[2022-06-21T23:06:08.850769-0400][CRITICAL][logger]][31540] > HANDLER Request <UID: 1251c278a3e24047ab7a026ce76c39b7>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.2805, 0.2078, 0.2659, -0.0736, -0.1933, 0.1437, -0.3040],    [-0.5771, -0.1384, -0.0928, 0.1353, -0.1181, 0.3780, 0.0328],    [ 0.2520, -0.1970, 0.2078, -0.3847, 0.0600, -0.0079, -0.1552],    [ 0.0747, -0.2829, 0.1337, 0.3658, -0.1159, 0.0539, 0.0621],    [-0.1712, 0.1728, -0.1325, -0.1952, 0.2034, 0.0551, 0.1286],    [-0.2561, 0.2618, -0.0456, -0.2311, 0.2943, -0.0321, 0.0985],    [ 0.2760, 0.1936, 0.2807, -0.4258, 0.0132, 0.1108, -0.1365],    [-0.0857, -0.3008, -0.0978, 0.5168, -0.0946, -0.0879, -0.1025],    [ 0.2193, -0.2837, 0.1380, -0.2875, -0.0462, -0.0602, 0.1090],    [ 0.2251, -0.2831, 0.2760, 0.1932, -0.2166, -0.0986, 0.1936]]))